## Web Scraping Lab 1:


### Prepare your project:

#### Business goal:

Make sure you've understood the big picture of your project: the goal of the company (Gnod), their current product (Gnoosic), their strategy, and how your project fits into this context. Re-read the business case and the e-mail from the CTO, take a look at the flowchart and create an initial Trello board with the tasks you think you'll have to acomplish.

#### Scraping popular songs:

Your product will take a song as an input from the user and will output another song (the recommendation). In most cases, the recommended song will have to be similar to the inputed song, but the CTO thinks that if the song is on the top charts at the moment, the user will enjoy more a recommendation of a song that's also popular at the moment.

You have find data on the internet about currently popular songs. Billboard mantains a weekly Top 100 of "hot" songs here: https://www.billboard.com/charts/hot-100. 

It's a good place to start! Scrape the current top 100 songs and their respective artists, and put the information into a pandas dataframe.

# Setup

In [51]:
# Import libraries
import requests # to download html code
from bs4 import BeautifulSoup # to navigate through the html code
import pandas as pd
import numpy as np
import re # for cleanup
import spotipy
from config import *
import json
from spotipy.oauth2 import SpotifyClientCredentials

In [52]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= Client_ID, client_secret=Client_Secret))

In [53]:
# url used for top100
url = "https://www.billboard.com/charts/hot-100"

In [54]:
# Download html
response = requests.get(url)
# 200 status code means OK! response.status_code
print(response.status_code)

200


In [55]:
# Parse html
soup = BeautifulSoup(response.text, 'html.parser')
# Check up
# soup

# Scraping

## Top 100

### Songname

In [56]:
# charts > div > div.chart-list__wrapper > div > ol > li:nth-child(1) > button > span.chart-element__information > span.chart-element__information__song.text--truncate.color--primary

In [57]:
songnametext = soup.select("span.chart-element__information__song")[0].get_text()
songnametext

'Montero (Call Me By Your Name)'

In [58]:
Songnames = []
for elem in soup.select("span.chart-element__information__song"):
    Songnames.append(elem.get_text())

print(Songnames)

['Montero (Call Me By Your Name)', 'Peaches', 'Leave The Door Open', 'Up', 'Drivers License', 'Save Your Tears', 'Levitating', 'Blinding Lights', 'Mood', 'What You Know Bout Love', 'Tombstone', 'Astronaut In The Ocean', "What's Next", '34+35', 'Go Crazy', 'Street Runner', 'Best Friend', 'Calling My Phone', 'Therefore I Am', 'Back In Blood', 'You Broke Me First.', 'Richer', "You're Mines Still", 'Beat Box', 'Positions', 'On Me', 'Dakiti', 'Heartbreak Anniversary', "My Ex's Best Friend", 'Dynamite', 'Without You', 'The Good Ones', 'Beautiful Mistakes', 'Wants And Needs', 'Streets', 'Anyone', 'For The Night', 'Whoopty', 'No More Parties', 'Good Days', 'Starting Over', "What's Your Country Song", 'Put Your Records On', 'Cry Baby', 'Long Live', 'Hold On', 'Track Star', "We're Good", 'Hard For The Next', 'Heat Waves', "You All Over Me (Taylor's Version) (From The Vault)", 'Lady', 'Telepatia', 'Forever After All', 'SoulFly', 'Goosebumps', 'My Head And My Heart', 'Time Today', 'Willow', 'Just 

### Artist

In [59]:
# charts > div > div.chart-list__wrapper > div > ol > li:nth-child(1) > button > span.chart-element__information > span.chart-element__information__artist.text--truncate.color--secondary

In [60]:
artisttext = soup.select("span.chart-element__information__artist")[0].get_text()
artisttext

'Lil Nas X'

In [61]:
Artists = []
for elem in soup.select("span.chart-element__information__artist"):
    Artists.append(elem.get_text())

print(Artists)

['Lil Nas X', 'Justin Bieber Featuring Daniel Caesar & Giveon', 'Silk Sonic (Bruno Mars & Anderson .Paak)', 'Cardi B', 'Olivia Rodrigo', 'The Weeknd', 'Dua Lipa Featuring DaBaby', 'The Weeknd', '24kGoldn Featuring iann dior', 'Pop Smoke', 'Rod Wave', 'Masked Wolf', 'Drake', 'Ariana Grande', 'Chris Brown & Young Thug', 'Rod Wave', 'Saweetie Featuring Doja Cat', 'Lil Tjay Featuring 6LACK', 'Billie Eilish', 'Pooh Shiesty Featuring Lil Durk', 'Tate McRae', 'Rod Wave Featuring Polo G', 'Yung Bleu Featuring Drake', 'SpotemGottem Featuring Pooh Shiesty Or DaBaby', 'Ariana Grande', 'Lil Baby', 'Bad Bunny & Jhay Cortez', 'Giveon', 'Machine Gun Kelly X blackbear', 'BTS', 'The Kid LAROI', 'Gabby Barrett', 'Maroon 5 Featuring Megan Thee Stallion', 'Drake Featuring Lil Baby', 'Doja Cat', 'Justin Bieber', 'Pop Smoke Featuring Lil Baby & DaBaby', 'CJ', 'Coi Leray Featuring Lil Durk', 'SZA', 'Chris Stapleton', 'Thomas Rhett', 'Ritt Momney', 'Megan Thee Stallion Featuring DaBaby', 'Florida Georgia Line

## Spotify

### Playlist 1

In [62]:
# https://open.spotify.com/playlist/1Ien12ACUL1Q2V2xIra3WO
playlist_1 = sp.user_playlist_tracks("spotify", "1Ien12ACUL1Q2V2xIra3WO")

In [63]:
type(playlist_1)

dict

In [64]:
print(list(playlist_1.keys())) # items and total songs
print("Total number of songs in the playlist: ",playlist_1["total"]) 
len(playlist_1["items"]) # It is limited to 100 tracks, we will have to fix it:

['href', 'items', 'limit', 'next', 'offset', 'previous', 'total']
Total number of songs in the playlist:  8846


100

In [65]:
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks


In [66]:
playlist = get_playlist_tracks("spotify", "1Ien12ACUL1Q2V2xIra3WO")


In [67]:
len(playlist)

8846

In [68]:
# playlist

In [69]:
playlist[0]['track']['name'] #songname

'beverly'

In [70]:
for artist in playlist[0]['track']['artists']: #artists name (can be more than 1)
    print(artist['name'])

Koi


In [71]:
playlist[0]['track']['uri'] #uri for audio features

'spotify:track:2pTW1GkqQ6dPTBOff2ZqQc'

In [72]:
song_names = []
song_uri = []
artist_names = []
audio_features = []
counter = 0
song_uri_total = []

for item in playlist:
    if item["is_local"] == False:   # is_local == True is stored locally?/duplicates
        counter = counter +1
    
        song_names.append(item['track']['name'])
        song_uri_total.append(item['track'] ['uri'])
        song_uri.append(item['track']['uri'])
    
        tempo_artists = []
        for artist in item['track']['artists']:
            tempo_artists.append(artist['name'])
        
        artist_names.append(", ".join(tempo_artists))
    
        if counter == 100:
            audio_features.append(sp.audio_features(song_uri))
            counter = 0
            song_uri = []      
      
audio_features.append(sp.audio_features(song_uri))  

In [73]:
audio_features_total = [subitem for item in audio_features for subitem in item]

In [74]:
len (song_names)

8845

In [75]:
len(audio_features_total)

8845

In [76]:
audi_features_df = pd.DataFrame(audio_features_total)
audi_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.735,0.491,0,-7.766,0,0.0427,0.15700,0.000060,0.121,0.5740,167.130,audio_features,2pTW1GkqQ6dPTBOff2ZqQc,spotify:track:2pTW1GkqQ6dPTBOff2ZqQc,https://api.spotify.com/v1/tracks/2pTW1GkqQ6dP...,https://api.spotify.com/v1/audio-analysis/2pTW...,103473,4
1,0.602,0.796,0,-3.657,0,0.1030,0.06820,0.120000,0.150,0.2650,126.060,audio_features,47Z5890IcjSed81ldeLgqc,spotify:track:47Z5890IcjSed81ldeLgqc,https://api.spotify.com/v1/tracks/47Z5890IcjSe...,https://api.spotify.com/v1/audio-analysis/47Z5...,245053,4
2,0.800,0.585,10,-7.343,1,0.0924,0.26400,0.000000,0.153,0.7790,126.058,audio_features,6PGoSes0D9eUDeeAafB2As,spotify:track:6PGoSes0D9eUDeeAafB2As,https://api.spotify.com/v1/tracks/6PGoSes0D9eU...,https://api.spotify.com/v1/audio-analysis/6PGo...,213400,4
3,0.747,0.592,8,-6.334,1,0.0457,0.00517,0.000015,0.124,0.1760,110.991,audio_features,3sTCfUmYXSVWDacTd6uMbQ,spotify:track:3sTCfUmYXSVWDacTd6uMbQ,https://api.spotify.com/v1/tracks/3sTCfUmYXSVW...,https://api.spotify.com/v1/audio-analysis/3sTC...,212027,3
4,0.341,0.193,2,-16.915,1,0.0364,0.84800,0.923000,0.131,0.0749,104.448,audio_features,2Twe7p278J2GxzjQZWJZWM,spotify:track:2Twe7p278J2GxzjQZWJZWM,https://api.spotify.com/v1/tracks/2Twe7p278J2G...,https://api.spotify.com/v1/audio-analysis/2Twe...,164107,4


In [77]:
audi_features_df = audi_features_df.drop(columns=['type', 'id', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])

In [78]:
audi_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,uri
0,0.735,0.491,0,-7.766,0,0.0427,0.15700,0.000060,0.121,0.5740,167.130,spotify:track:2pTW1GkqQ6dPTBOff2ZqQc
1,0.602,0.796,0,-3.657,0,0.1030,0.06820,0.120000,0.150,0.2650,126.060,spotify:track:47Z5890IcjSed81ldeLgqc
2,0.800,0.585,10,-7.343,1,0.0924,0.26400,0.000000,0.153,0.7790,126.058,spotify:track:6PGoSes0D9eUDeeAafB2As
3,0.747,0.592,8,-6.334,1,0.0457,0.00517,0.000015,0.124,0.1760,110.991,spotify:track:3sTCfUmYXSVWDacTd6uMbQ
4,0.341,0.193,2,-16.915,1,0.0364,0.84800,0.923000,0.131,0.0749,104.448,spotify:track:2Twe7p278J2GxzjQZWJZWM


In [79]:
spotify_names = pd.DataFrame({'Songtitle': song_names, 'Artist(s)': artist_names, 'uri': song_uri_total})

In [80]:
spotify_names

,Songtitle,Artist(s),uri
0,beverly,Koi,spotify:track:2pTW1GkqQ6dPTBOff2ZqQc
1,Titanium (feat. Sia),"David Guetta, Sia",spotify:track:47Z5890IcjSed81ldeLgqc
2,LOVE. FEAT. ZACARI.,"Kendrick Lamar, Zacari",spotify:track:6PGoSes0D9eUDeeAafB2As
3,do re mi,blackbear,spotify:track:3sTCfUmYXSVWDacTd6uMbQ
4,Lord Of The Rings: The Fellowship Of The Ring ...,"Raine, The City of Prague Philharmonic Orchest...",spotify:track:2Twe7p278J2GxzjQZWJZWM
...,...,...,...
8840,Disco Man,Remi Wolf,spotify:track:0T7aTl1t15HKHfwep4nANV
8841,"Chillin Vibe (Feat. Chaboom, kitsyojii, Wonjja...","YoBoy, Chaboom, kitsyojii, 원쨩, Leebido, 오아이, k...",spotify:track:6wXNvp1ZAm13IHAsU0AxsH
8842,Run It Up (feat. Offset & Moneybagg Yo),"Lil Tjay, Offset, Moneybagg Yo",spotify:track:5pmvv42h1bcK3frE8CEblt
8843,Headshot (feat. Polo G & Fivio Foreign),"Lil Tjay, Polo G, Fivio Foreign",spotify:track:05bcw8n2kOlB7jpaXC41zo


### Playlist 2

In [81]:
 # https://open.spotify.com/playlist/081RobLq4ExzALJVAWLmkk
playlist_2 = sp.user_playlist_tracks("spotify", "081RobLq4ExzALJVAWLmkk")

In [82]:
print(list(playlist_2.keys())) # items and total songs
print("Total number of songs in the playlist: ",playlist_2["total"]) 
len(playlist_2["items"]) # It is limited to 100 tracks, we will have to fix it:

['href', 'items', 'limit', 'next', 'offset', 'previous', 'total']
Total number of songs in the playlist:  9983


100

In [83]:
playlist_2 = get_playlist_tracks("spotify", "081RobLq4ExzALJVAWLmkk")

In [84]:
len(playlist_2)

9983

In [85]:
song_names = []
song_uri = []
artist_names = []
audio_features = []
counter = 0
song_uri_total = []

for item in playlist_2:
    if item["is_local"] == False:   # is_local == True is stored locally?/duplicates
        counter = counter +1
    
        song_names.append(item['track']['name'])
        song_uri_total.append(item['track'] ['uri'])
        song_uri.append(item['track']['uri'])
    
        tempo_artists = []
        for artist in item['track']['artists']:
            tempo_artists.append(artist['name'])
        
        artist_names.append(", ".join(tempo_artists))
    
        if counter == 100:
            audio_features.append(sp.audio_features(song_uri))
            counter = 0
            song_uri = []      
      
audio_features.append(sp.audio_features(song_uri))  

In [86]:
audio_features_total = [subitem for item in audio_features for subitem in item]

In [87]:
len (song_names)

9983

In [88]:
len(audio_features_total)

9983

In [89]:
audi_features_df_2 = pd.DataFrame(audio_features_total)
audi_features_df_2.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.626,0.800,1,-3.988,0,0.0350,0.0267,0.000138,0.0623,0.258,127.966,audio_features,0DhkTGjEaIHaSxTfC14BFm,spotify:track:0DhkTGjEaIHaSxTfC14BFm,https://api.spotify.com/v1/tracks/0DhkTGjEaIHa...,https://api.spotify.com/v1/audio-analysis/0Dhk...,160184,4
1,0.691,0.509,6,-9.353,1,0.0459,0.4610,0.000003,0.1460,0.268,86.101,audio_features,74lxmxZDGDr21dKyXU0rcr,spotify:track:74lxmxZDGDr21dKyXU0rcr,https://api.spotify.com/v1/tracks/74lxmxZDGDr2...,https://api.spotify.com/v1/audio-analysis/74lx...,215200,4
2,0.756,0.743,5,-6.915,1,0.0968,0.0925,0.000290,0.0616,0.413,105.010,audio_features,5wmlyNmVGwE3Y3TKKPFNpF,spotify:track:5wmlyNmVGwE3Y3TKKPFNpF,https://api.spotify.com/v1/tracks/5wmlyNmVGwE3...,https://api.spotify.com/v1/audio-analysis/5wml...,208065,4
3,0.555,0.137,4,-13.874,0,0.0445,0.9630,0.000010,0.1210,0.205,137.798,audio_features,3aN15iceA1SfExPEs6kfLj,spotify:track:3aN15iceA1SfExPEs6kfLj,https://api.spotify.com/v1/tracks/3aN15iceA1Sf...,https://api.spotify.com/v1/audio-analysis/3aN1...,238708,4
4,0.486,0.854,0,-4.859,1,0.0569,0.0739,0.000011,0.3580,0.469,123.942,audio_features,38tGRxFP5NkWbtOWsXbQ0L,spotify:track:38tGRxFP5NkWbtOWsXbQ0L,https://api.spotify.com/v1/tracks/38tGRxFP5NkW...,https://api.spotify.com/v1/audio-analysis/38tG...,208548,4


In [90]:
audi_features_df_2 = audi_features_df_2.drop(columns=['type', 'id', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])

In [91]:
audi_features_df_2.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,uri
0,0.626,0.800,1,-3.988,0,0.0350,0.0267,0.000138,0.0623,0.258,127.966,spotify:track:0DhkTGjEaIHaSxTfC14BFm
1,0.691,0.509,6,-9.353,1,0.0459,0.4610,0.000003,0.1460,0.268,86.101,spotify:track:74lxmxZDGDr21dKyXU0rcr
2,0.756,0.743,5,-6.915,1,0.0968,0.0925,0.000290,0.0616,0.413,105.010,spotify:track:5wmlyNmVGwE3Y3TKKPFNpF
3,0.555,0.137,4,-13.874,0,0.0445,0.9630,0.000010,0.1210,0.205,137.798,spotify:track:3aN15iceA1SfExPEs6kfLj
4,0.486,0.854,0,-4.859,1,0.0569,0.0739,0.000011,0.3580,0.469,123.942,spotify:track:38tGRxFP5NkWbtOWsXbQ0L


In [92]:
spotify_names_2 = pd.DataFrame({'Songtitle': song_names, 'Artist(s)': artist_names, 'uri': song_uri_total})

In [93]:
spotify_names_2

,Songtitle,Artist(s),uri
0,Lullaby,"R3HAB, Mike Williams",spotify:track:0DhkTGjEaIHaSxTfC14BFm
1,Che vuoi che sia,Irama,spotify:track:74lxmxZDGDr21dKyXU0rcr
2,Din Krigare,"Cluee, Rabih",spotify:track:5wmlyNmVGwE3Y3TKKPFNpF
3,Lullaby,Sara Mancuso,spotify:track:3aN15iceA1SfExPEs6kfLj
4,Old Guitar,BUNT.,spotify:track:38tGRxFP5NkWbtOWsXbQ0L
...,...,...,...
9978,Make It Right,Denvre,spotify:track:071xJT5GV6CCyWKXQpGx9B
9979,Hometown,Cailee Rae,spotify:track:7KhfYu171bETqDqV262jwE
9980,waves,renforshort,spotify:track:76gK9am1WZ3BwO73xnAOcf
9981,SAVIOR,Elley Duhé,spotify:track:60uwjx3zc93kVyUtNxVOsi


# Dataframe

## Top 100

In [94]:
Top_100 = pd.DataFrame({'Songtitle': Songnames, 'Artist(s)': Artists})
Top_100

,Songtitle,Artist(s)
0,Montero (Call Me By Your Name),Lil Nas X
1,Peaches,Justin Bieber Featuring Daniel Caesar & Giveon
2,Leave The Door Open,Silk Sonic (Bruno Mars & Anderson .Paak)
3,Up,Cardi B
4,Drivers License,Olivia Rodrigo
...,...,...
95,Shock Da World,Rod Wave
96,You Got It,VEDO
97,Sneaky Links,Rod Wave
98,Nobody,Dylan Scott


## Spotify

In [95]:
audi_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,uri
0,0.735,0.491,0,-7.766,0,0.0427,0.15700,0.000060,0.121,0.5740,167.130,spotify:track:2pTW1GkqQ6dPTBOff2ZqQc
1,0.602,0.796,0,-3.657,0,0.1030,0.06820,0.120000,0.150,0.2650,126.060,spotify:track:47Z5890IcjSed81ldeLgqc
2,0.800,0.585,10,-7.343,1,0.0924,0.26400,0.000000,0.153,0.7790,126.058,spotify:track:6PGoSes0D9eUDeeAafB2As
3,0.747,0.592,8,-6.334,1,0.0457,0.00517,0.000015,0.124,0.1760,110.991,spotify:track:3sTCfUmYXSVWDacTd6uMbQ
4,0.341,0.193,2,-16.915,1,0.0364,0.84800,0.923000,0.131,0.0749,104.448,spotify:track:2Twe7p278J2GxzjQZWJZWM


In [96]:
spotify_names.head()

,Songtitle,Artist(s),uri
0,beverly,Koi,spotify:track:2pTW1GkqQ6dPTBOff2ZqQc
1,Titanium (feat. Sia),"David Guetta, Sia",spotify:track:47Z5890IcjSed81ldeLgqc
2,LOVE. FEAT. ZACARI.,"Kendrick Lamar, Zacari",spotify:track:6PGoSes0D9eUDeeAafB2As
3,do re mi,blackbear,spotify:track:3sTCfUmYXSVWDacTd6uMbQ
4,Lord Of The Rings: The Fellowship Of The Ring ...,"Raine, The City of Prague Philharmonic Orchest...",spotify:track:2Twe7p278J2GxzjQZWJZWM


In [97]:
Spotify_df = pd.merge(spotify_names, audi_features_df, how="inner", on=["uri", "uri"])

In [98]:
Spotify_df.head()

,Songtitle,Artist(s),uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,beverly,Koi,spotify:track:2pTW1GkqQ6dPTBOff2ZqQc,0.735,0.491,0,-7.766,0,0.0427,0.15700,0.000060,0.121,0.5740,167.130
1,Titanium (feat. Sia),"David Guetta, Sia",spotify:track:47Z5890IcjSed81ldeLgqc,0.602,0.796,0,-3.657,0,0.1030,0.06820,0.120000,0.150,0.2650,126.060
2,LOVE. FEAT. ZACARI.,"Kendrick Lamar, Zacari",spotify:track:6PGoSes0D9eUDeeAafB2As,0.800,0.585,10,-7.343,1,0.0924,0.26400,0.000000,0.153,0.7790,126.058
3,do re mi,blackbear,spotify:track:3sTCfUmYXSVWDacTd6uMbQ,0.747,0.592,8,-6.334,1,0.0457,0.00517,0.000015,0.124,0.1760,110.991
4,Lord Of The Rings: The Fellowship Of The Ring ...,"Raine, The City of Prague Philharmonic Orchest...",spotify:track:2Twe7p278J2GxzjQZWJZWM,0.341,0.193,2,-16.915,1,0.0364,0.84800,0.923000,0.131,0.0749,104.448


In [99]:
Spotify_2_df = pd.merge(spotify_names_2, audi_features_df_2, how="inner", on=["uri", "uri"])

In [100]:
Spotify_2_df.head()

,Songtitle,Artist(s),uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Lullaby,"R3HAB, Mike Williams",spotify:track:0DhkTGjEaIHaSxTfC14BFm,0.626,0.800,1,-3.988,0,0.0350,0.0267,0.000138,0.0623,0.258,127.966
1,Che vuoi che sia,Irama,spotify:track:74lxmxZDGDr21dKyXU0rcr,0.691,0.509,6,-9.353,1,0.0459,0.4610,0.000003,0.1460,0.268,86.101
2,Din Krigare,"Cluee, Rabih",spotify:track:5wmlyNmVGwE3Y3TKKPFNpF,0.756,0.743,5,-6.915,1,0.0968,0.0925,0.000290,0.0616,0.413,105.010
3,Lullaby,Sara Mancuso,spotify:track:3aN15iceA1SfExPEs6kfLj,0.555,0.137,4,-13.874,0,0.0445,0.9630,0.000010,0.1210,0.205,137.798
4,Old Guitar,BUNT.,spotify:track:38tGRxFP5NkWbtOWsXbQ0L,0.486,0.854,0,-4.859,1,0.0569,0.0739,0.000011,0.3580,0.469,123.942


In [101]:
result = Spotify_df.append(Spotify_2_df, sort=False)

In [ ]:
result.to_csv(r'../Gnoosic/Spotify.csv', sep=',', index=False)